# 01 - Explorar proyecto

Usamos la **Resources API** para listar topics, filtros, canales, tags y vistas (dashboards/reports) de un proyecto. Necesitas un **project_id** de tu cuenta (o usa uno devuelto por el notebook 00).

## 1. Configuración

In [ ]:
# @title Credenciales { display-mode: "form" }
ACCESS_TOKEN = ""  # @param {type:"string"}
PROJECT_ID = ""     # @param {type:"string"}

BASE_URL = "https://api.talkwalker.com"
import requests
import json

def tw_get(endpoint, params=None):
    p = {"access_token": ACCESS_TOKEN}
    if params:
        p.update(params)
    r = requests.get(f"{BASE_URL}{endpoint}", params=p)
    r.raise_for_status()
    return r.json()

def show(data):
    print(json.dumps(data, indent=2, ensure_ascii=False))

assert ACCESS_TOKEN, "Configura ACCESS_TOKEN"
assert PROJECT_ID, "Configura PROJECT_ID"
print("✅ Listo. Proyecto:", PROJECT_ID)

## 2. Listar topics y filtros

In [ ]:
resources = tw_get(f"/api/v2/talkwalker/p/{PROJECT_ID}/resources", params={"type": "search,filter"})
if resources.get("status_code") != "0":
    print("Error:", resources.get("status_message"))
else:
    for proj in resources.get("result_resources", {}).get("projects", []):
        print("Topics:")
        for cat in proj.get("topics", []):
            for node in cat.get("nodes", []):
                print(f"  - {node.get('id')}: {node.get('title')}")
        print("Filters:")
        for f in proj.get("filters", []):
            for node in f.get("nodes", []):
                print(f"  - {node.get('id')}: {node.get('title')}")

## 3. Listar canales / páginas monitoreadas

In [ ]:
pages = tw_get(f"/api/v2/talkwalker/p/{PROJECT_ID}/resources", params={"type": "page"})
if pages.get("status_code") == "0":
    for proj in pages.get("result_resources", {}).get("projects", []):
        for ch in proj.get("channels", []):
            for node in ch.get("nodes", []):
                print(f"  - {node.get('id')}: {node.get('title')}")
else:
    print("Error:", pages.get("status_message"))

## 4. Listar tags del proyecto

In [ ]:
tags_resp = tw_get(f"/api/v2/talkwalker/p/{PROJECT_ID}/tags")
if tags_resp.get("status_code") == "0":
    for t in tags_resp.get("result_tags", {}).get("tags", []):
        print(f"  - {t.get('id')}: {t.get('label')}")
else:
    print("Error:", tags_resp.get("status_message"))

## 5. Listar dashboards, reports y alerts

In [ ]:
views = tw_get(f"/api/v2/talkwalker/p/{PROJECT_ID}/views")
if views.get("status_code") == "0":
    for proj in views.get("result_views", {}).get("projects", []):
        for d in proj.get("dashboards", []):
            print(f"Dashboard: {d.get('id')} - {d.get('title')}")
        for r in proj.get("reports", []):
            print(f"Report/Alert: {r.get('id')} - {r.get('title')}")
else:
    print("Error:", views.get("status_message"))